In [1]:
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
def order_book(month,day):
    data1 = []
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/stat_order_book/CNF14_0'+str(month)+'_'+str(day)+'_order_book_final.csv'
    data1 = pd.read_csv(datapath,sep='\t',encoding = 'utf-8')
    data_book = data1[['0','1','2','3']]
    return data_book

In [3]:
def day_time(month,day):
    data = []
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_2014.0'+str(month)+'.'+str(day)+'.csv'
    data = pd.read_csv(datapath)
    data_CNF14 = data[data.Series == 'CNF14']
    data = data_CNF14
    market_open_time = data[data['TimeStamp'].str.contains('2014-0'+str(month)+'-'+str(day)+'D09:00')].index.tolist()[0] 
    market_close_time = data[data['TimeStamp'].str.contains('2014-0'+str(month)+'-'+str(day)+'D16:00')].index.tolist()[0] 
    data_open = data[market_open_time:market_close_time+1]
    timestamp_ = data_open.TimeStamp.unique()
    return timestamp_

In [4]:
def time_transform(timestamp_time):
    time_second = []
    for i in range(0,len(timestamp_time),1):
        second = float(timestamp_time[i][11])*36000 + float(timestamp_time[i][12])*3600 \
                +float(timestamp_time[i][14])*600 + float(timestamp_time[i][15])*60\
                +float(timestamp_time[i][17])*10 + float(timestamp_time[i][18])  
        time_second.append(second - 32400.0)
    return  time_second

In [5]:
def bid123_ask123_Q(data_book_28_open):
    Bid1 = []
    Bid2 = []
    Bid3 = []
    Bid1_Quantity = []
    Bid2_Quantity = []
    Bid3_Quantity = []
    Ask1 = []
    Ask2 = []
    Ask3 = []
    Ask1_Quantity = []
    Ask2_Quantity = []
    Ask3_Quantity = []

    TimeStamp = []
    for i in range(1,len(data_book_28_open),4):
        #print data_book_28_open.iloc[i]['0']
        #print data_book_28_open.iloc[i]['2']
        Bid1.append(float(data_book_28_open.iloc[i]['0'])/100.0)
        Bid1_Quantity.append(float(data_book_28_open.iloc[i]['1']))

        Bid2.append(float(data_book_28_open.iloc[i + 1]['0'])/100.0)
        Bid2_Quantity.append(float(data_book_28_open.iloc[i + 1]['1']))  

        Bid3.append(float(data_book_28_open.iloc[i + 2]['0'])/100.0)
        Bid3_Quantity.append(float(data_book_28_open.iloc[i + 2]['1']))  

        Ask1.append(float(data_book_28_open.iloc[i]['2'])/100.0) 
        Ask1_Quantity.append(float(data_book_28_open.iloc[i]['3']))

        Ask2.append(float(data_book_28_open.iloc[i + 1]['2'])/100.0) 
        Ask2_Quantity.append(float(data_book_28_open.iloc[i + 1]['3']))    

        Ask3.append(float(data_book_28_open.iloc[i + 2]['2'])/100.0) 
        Ask3_Quantity.append(float(data_book_28_open.iloc[i + 2]['3']))    
        TimeStamp.append(data_book_28_open.iloc[i-1][1])
    return Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity,    

In [6]:
def rise_bid(Bid1,timestamp_time_second):
    rise_ratio = []
    index = np.where(np.array(timestamp_time_second) >= 600)[0][0]
    for i in range(0,index):
        rise_ratio_ = round((Bid1[i] - Bid1[0])*(1.0)/Bid1[0]*100,5)
        rise_ratio.append(rise_ratio_)
    for i in range(index,len(Bid1),1):
        index_start = np.where(np.array(timestamp_time_second[:i]) >= timestamp_time_second[i] - 600)[0][0]
        rise_ratio_ = round((Bid1[i] - Bid1[index_start])*(1.0)/Bid1[index_start]*100,5)
        rise_ratio.append(rise_ratio_)
    return rise_ratio

In [7]:
def rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,close_price,\
             N_1,N_2,N_3,N_4,N_5,N_1_T,N_2_T,N_3_T,N_4_T,N_5_T,\
             N_1_E,N_2_E,N_3_E,N_4_E,N_5_E,N_1_M,N_2_M,N_3_M,N_4_M,N_5_M,\
             N_1_TI,N_2_TI,N_3_TI,N_4_TI,N_5_TI):
    
    if rise_ratio_bid[index] <= -0.4:
        N_1 = N_1 + 1
        #print rise_ratio_ask[index]
        #Ask1[i] < max_Bid1_time_series[i+1]
        if Ask[index] < max(Bid[(index+1):index + index_15_min + 1]):
            N_1_T = N_1_T + 1
            N_1_E = N_1_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_1_E
            first_max_index = np.where(np.array(Bid[(index + 1):index + 1 + index_15_min + 1 ]) > Ask[index])[0][0]
            N_1_M = N_1_M + (Ask[index] - min(Bid[(index + 1):(index + 1 + first_max_index + 1)]))*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_max_index + 1]
            N_1_TI = N_1_TI + (time_traded - time_start)
            #print N_1_TI
            pass
        elif Ask[index] >= max(Bid[(index+1):index + index_15_min + 1]):
            if i < 25200 - 900:
                N_1_E = N_1_E + (Bid[index + index_15_min] - Bid[index])*(1.0)/(5.0)
                N_1_M = N_1_M + (Ask[index] - min(Bid[(index+1):(index + 1 + index_15_min + 1 )]))*(1.0)/(5.0) 
                #print N_1_M
                N_1_TI = N_1_TI + (900.0)
                #print N_1_TI
            elif i > 25200 - 900:
                N_1_E = N_1_E + (close_price - Bid[index])*(1.0)/(5.0)
                N_1_M = N_1_M + (Ask[index] - min(Bid[(index+1):]))*(1.0)/(5.0) 
                time_start = timestamp_time_second[index]
                time_traded = 25200              
                N_1_TI = N_1_TI + (time_traded - time_start)
                #print N_1_TI
                pass
            pass

    elif -0.4 < rise_ratio_bid[index] <= -0.15:
        N_2 = N_2 + 1
        #print rise_ratio_ask[index]
        if Ask[index] < max(Bid[(index+1):index + index_15_min + 1]):
            #print 3
            N_2_T = N_2_T + 1
            N_2_E = N_2_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_2_E
            first_max_index = np.where(np.array(Bid[(index+1):index + 1 + index_15_min + 1 ]) > Ask[index])[0][0]
            N_2_M = N_2_M + (Ask[index] - min(Bid[(index + 1):(index + 1 + first_max_index + 1)]))*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_max_index + 1]
            N_2_TI = N_2_TI + (time_traded - time_start)
            #print N_2_TI
            pass
        elif Ask[index] >= max(Bid[(index+1):index + index_15_min + 1]):
            if i < 25200 - 900:
                N_2_E = N_2_E + (Bid[index + index_15_min] - Bid[index])*(1.0)/(5.0)
                N_2_M = N_2_M + (Ask[index] - min(Bid[(index+1):(index + 1 + index_15_min + 1 )]))*(1.0)/(5.0) 
                N_2_TI = N_2_TI + (900.0)
                #print N_2_TI
            elif i >= 25200 - 900:
                N_2_E = N_2_E + (close_price - Bid[index])*(1.0)/(5.0)
                N_2_M = N_2_M + (Ask[index] - min(Bid[(index+1):]))*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = 25200               
                N_2_TI = N_2_TI + (time_traded - time_start)
                #print N_2_TI
                pass
            pass

    elif -0.15 < rise_ratio_bid[index] <= 0.15:
        N_3 = N_3 + 1
        #print rise_ratio_ask[index]
        if Ask[index] < max(Bid[(index+1):index + index_15_min + 1]):
            #print 5
            N_3_T = N_3_T + 1
            N_3_E = N_3_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_3_E
            first_max_index = np.where(np.array(Bid[(index + 1):index + 1 + index_15_min + 1]) > Ask[index])[0][0]
            N_3_M = N_3_M + (Ask[index] - min(Bid[(index + 1):(index + 1 + first_max_index + 1)]))*(1.0)/(5.0) 
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_max_index + 1]
            N_3_TI = N_3_TI + (time_traded - time_start)
            #print N_3_TI
            pass
        elif Ask[index] >= max(Bid[(index+1):index + index_15_min + 1]):
            if i < 25200 - 900:
                N_3_E = N_3_E + (Bid[index + index_15_min] - Bid[index])*(1.0)/(5.0)
                N_3_M = N_3_M + (Ask[index] - min(Bid[(index+1):(index + 1 + index_15_min + 1 )]))*(1.0)/(5.0) 
                N_3_TI = N_3_TI + (900.0)
                #print N_3_TI
            elif i >= 25200 - 900:
                N_3_E = N_3_E + (close_price - Bid[index])*(1.0)/(5.0)
                N_3_M = N_3_M + (Ask[index] - min(Bid[(index+1):]))*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = 25200              
                N_3_TI = N_3_TI + (time_traded - time_start)
                #print N_3_TI
                pass
            pass

    elif 0.15 < rise_ratio_bid[index] < 0.4:
        N_4 = N_4 + 1
        #print rise_ratio_ask[index]
        if Ask[index] < max(Bid[(index+1):index + index_15_min + 1]):
            #print 7
            N_4_T = N_4_T + 1
            N_4_E = N_4_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_4_E
            first_max_index = np.where(np.array(Bid[(index+1):index + 1 + index_15_min + 1 ]) > Ask[index])[0][0]
            N_4_M = N_4_M + (Ask[index] - min(Bid[(index + 1):(index + 1 + first_max_index + 1)]))*(1.0)/(5.0) 
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_max_index + 1]
            N_4_TI = N_4_TI + (time_traded - time_start)
            #print N_4_TI
            pass
        else:
            if i < 25200 - 900:
                N_4_E = N_4_E + (Bid[index + index_15_min] - Bid[index])*(1.0)/(5.0)
                N_4_M = N_4_M + (Ask[index] - min(Bid[(index+1):(index + 1 + index_15_min + 1 )]))*(1.0)/(5.0) 
                N_4_TI = N_4_TI + (900.0)
                #print N_4_TI
            elif i >= 25200 - 900:
                N_4_E = N_4_E + (close_price - Bid[index])*(1.0)/(5.0)
                N_4_M = N_4_M + (Ask[index] - min(Bid[(index+1):]))*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = 25200              
                N_4_TI = N_4_TI + (time_traded - time_start)
                #print N_4_TI
                pass
            pass
    elif 0.4 <= rise_ratio_bid[index]:
        N_5 = N_5 + 1
        #print rise_ratio_ask[index]
        if Ask[index] < max(Bid[(index+1):index + index_15_min + 1]):
            N_5_T = N_5_T + 1
            N_5_E = N_5_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_5_E      
            first_max_index = np.where(np.array(Bid[(index+1):index + 1 + index_15_min + 1 ]) > Ask[index])[0][0]
            N_5_M = N_5_M + (Ask[index] - min(Bid[(index + 1):(index + 1 + first_max_index + 1)]))*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_max_index + 1]
            N_5_TI = N_5_TI + (time_traded - time_start)
            #print N_5_TI
            pass
        elif Ask[index] >= max(Bid[(index+1):index + index_15_min + 1]):
            if i < 25200 - 900:
                N_5_E = N_5_E + (Bid[index + index_15_min] - Bid[index])*(1.0)/(5.0)
                N_5_M = N_5_M + (Ask[index] - min(Bid[(index+1):(index + 1 + index_15_min + 1)]))*(1.0)/(5.0) 
                N_1_TI = N_1_TI + (900.0)
                #print N_5_TI
            elif i >= 25200 - 900:
                N_5_E = N_5_E + (close_price - Bid[index])*(1.0)/(5.0)
                N_5_M = N_5_M + (Ask[index] - min(Bid[(index+1):]))*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = 25200             
                N_5_TI = N_5_TI + (time_traded - time_start)
                #print N_5_TI 
                pass
            pass
        
    return N_1,N_2,N_3,N_4,N_5,N_1_T,N_2_T,N_3_T,N_4_T,N_5_T,N_1_E,N_2_E,N_3_E,N_4_E,N_5_E,N_1_M,N_2_M,N_3_M,N_4_M,N_5_M,\
           N_1_TI,N_2_TI,N_3_TI,N_4_TI,N_5_TI

In [8]:
def ask(time1,time2,Bid,Ask,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,\
        Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weigth1,weigth2,weigth3):
    [N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,\
     N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI] = [0 for i in  range(0,25,1)]
    [N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,\
     N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI] = [0 for i in  range(0,25,1)]
    [N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,\
     N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI] = [0 for i in  range(0,25,1)]
    [N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,\
     N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI] = [0 for i in  range(0,25,1)]
    [N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,\
     N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI] = [0 for i in  range(0,25,1)]

    for i in range(time1,time2,1):
        index = np.where(array(timestamp_time_second) <= i)[0][-1]
        if i < 25200 - 900:
            index_15_min = np.where(np.array(timestamp_time_second[index:]) >= i + 900)[0][0]

        weighted_Bid = Bid1_Quantity[index] * weight1 + Bid2_Quantity[index] * weight2 + Bid3_Quantity[index] * weight3
        weighted_Ask = Ask1_Quantity[index] * weight1 + Ask2_Quantity[index] * weight2 + Ask3_Quantity[index] * weight3  
        if (weighted_Bid*(1.0) / weighted_Ask) <= 0.7:
            N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,\
            N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI = rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,\
                                                          N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,\
                                                          N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI)                                               
        elif 0.7 < (weighted_Bid*(1.0) / weighted_Ask) <= 0.9:
            N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,\
            N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI = rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,\
                                                          N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,\
                                                          N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI)                                                
        elif 0.9 < (weighted_Bid*(1.0) / weighted_Ask) <= 1.1:
            N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,\
            N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI = rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,\
                                                          N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,\
                                                          N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI)                                    
        elif 1.1 < (weighted_Bid*(1.0) / weighted_Ask) <= 1.3:
            N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,\
            N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI = rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,\
                                                          N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,\
                                                          N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI)                              
        elif 1.30 < (weighted_Bid*(1.0) / weighted_Ask):
            N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,\
            N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI = rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,\
                                                          N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,\
                                                          N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI)                 
    return [[N_11,N_12,N_13,N_14,N_15],[N_11_T,N_12_T,N_13_T,N_14_T,N_15_T],[N_11_E,N_12_E,N_13_E,N_14_E,N_15_E],\
             [N_11_M,N_12_M,N_13_M,N_14_M,N_15_M],[N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI],\
            [N_21,N_22,N_23,N_24,N_25],[N_21_T,N_22_T,N_23_T,N_24_T,N_25_T],[N_21_E,N_22_E,N_23_E,N_24_E,N_25_E],\
             [N_21_M,N_22_M,N_23_M,N_24_M,N_25_M],[N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI],\
            [N_31,N_32,N_33,N_34,N_35],[N_31_T,N_32_T,N_33_T,N_34_T,N_35_T],[N_31_E,N_32_E,N_33_E,N_34_E,N_35_E],\
             [N_31_M,N_32_M,N_33_M,N_34_M,N_35_M],[N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI],\
            [N_41,N_42,N_43,N_44,N_45],[N_41_T,N_42_T,N_43_T,N_44_T,N_45_T],[N_41_E,N_42_E,N_43_E,N_44_E,N_45_E],\
             [N_41_M,N_42_M,N_43_M,N_44_M,N_45_M],[N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI],\
            [N_51,N_52,N_53,N_54,N_55],[N_51_T,N_52_T,N_53_T,N_54_T,N_55_T],[N_51_E,N_52_E,N_53_E,N_54_E,N_55_E],\
             [N_51_M,N_52_M,N_53_M,N_54_M,N_55_M],[N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI]] 




### Day 1/15

In [9]:
data_book = order_book(1,15)
data_book_open = data_book[1108:275559+1]
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,15)
timestamp_time_second = time_transform(timestamp_time)

In [10]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 104.846992


In [11]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 74.060694


In [12]:
close_price = 6740
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [13]:
time1 = 0
time2 = 600
mat28_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [20]:
#mat28_3_15 

In [14]:
time1 = 600
time2 = 7200
mat28_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [15]:
#mat28_3_11

In [15]:
time1 = 7200
time2 = 14400
mat28_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [16]:
time1 = 14400
time2 = 25200
mat28_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [69]:
#mat28_3_16

### Day 1/14

In [17]:
data_book = order_book(1,14)
data_book_open = data_book[880:336307+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,14)
timestamp_time_second = time_transform(timestamp_time)

In [18]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 167.486132


In [19]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 151.127659


In [20]:
close_price = 6780
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [21]:
time1 = 0
time2 = 600
mat29_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [48]:
#mat29_3_15

In [22]:
time1 = 600
time2 = 7200
mat29_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [23]:
time1 = 7200
time2 = 14400
mat29_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [24]:
time1 = 14400
time2 = 25200
mat29_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [25]:
#mat29_3_16

### Day 1/13

In [26]:
data_book = order_book(1,13)
data_book_open = data_book[1136:331583+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,13)
timestamp_time_second = time_transform(timestamp_time)

In [27]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 149.952572


In [28]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 141.576651


In [29]:
close_price = 6740
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [30]:
time1 = 0
time2 = 600
mat27_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [31]:
time1 = 600
time2 = 7200
mat27_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [32]:
time1 = 7200
time2 = 14400
mat27_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [33]:
time1 = 14400
time2 = 25200
mat27_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [97]:
#mat27_2_16

### Day 1/10

In [34]:
data_book = order_book(1,10)
data_book_open = data_book[904:351239+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,10)
timestamp_time_second = time_transform(timestamp_time)

In [35]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 143.389477


In [36]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 387.690488


In [37]:
close_price = 6760
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [38]:
time1 = 0
time2 = 600
mat24_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [39]:
time1 = 600
time2 = 7200
mat24_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [40]:
time1 = 7200
time2 = 14400
mat24_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [41]:
time1 = 14400
time2 = 25200
mat24_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/09

In [42]:
data_book = order_book(1,'09')
data_book_open = data_book[856:366355+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'09')
timestamp_time_second = time_transform(timestamp_time)

In [43]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 168.374224


In [44]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 403.099445


In [45]:
close_price = 6770
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [46]:
time1 = 0
time2 = 600
mat22_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [47]:
time1 = 600
time2 = 7200
mat22_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [48]:
time1 = 7200
time2 = 14400
mat22_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [49]:
time1 = 14400
time2 = 25200
mat22_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [128]:
#mat22_3_15

### Day 1/08

In [50]:
data_book = order_book(1,'08')
data_book_open = data_book[1236:356399+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'08')
timestamp_time_second = time_transform(timestamp_time)

In [51]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 169.020950


In [52]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 487.949712


In [53]:
close_price = 6810
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [54]:
time1 = 0
time2 = 600
mat21_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [55]:
time1 = 600
time2 = 7200
mat21_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [56]:
time1 = 7200
time2 = 14400
mat21_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [57]:
time1 = 14400
time2 = 25200
mat21_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/07

In [58]:
data_book = order_book(1,'07')
data_book_open = data_book[1040:325879+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'07')
timestamp_time_second = time_transform(timestamp_time)

In [59]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 113.045803


In [60]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 354.429260


In [61]:
close_price = 6790
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [62]:
time1 = 0
time2 = 600
mat20_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [63]:
time1 = 600
time2 = 7200
mat20_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [64]:
time1 = 7200
time2 = 14400
mat20_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [65]:
time1 = 14400
time2 = 25200
mat20_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/06

In [66]:
data_book = order_book(1,'06')
data_book_open = data_book[800:332843+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'06')
timestamp_time_second = time_transform(timestamp_time)

In [67]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 115.696991


In [68]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 127.249800


In [69]:
close_price = 6806
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [70]:
time1 = 0
time2 = 600
mat17_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [71]:
time1 = 600
time2 = 7200
mat17_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [72]:
time1 = 7200
time2 = 14400
mat17_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [73]:
time1 = 14400
time2 = 25200
mat17_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat17_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/03

In [74]:
data_book = order_book(1,'03')
data_book_open = data_book[984:318387+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'03')
timestamp_time_second = time_transform(timestamp_time)

In [75]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 110.710095


In [76]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 315.078904


In [77]:
close_price = 6915
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [78]:
time1 = 0
time2 = 600
mat16_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [79]:
time1 = 600
time2 = 7200
mat16_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [80]:
time1 = 7200
time2 = 14400
mat16_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [81]:
time1 = 14400
time2 = 25200
mat16_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat16_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 1/02

In [82]:
data_book = order_book(1,'02')
data_book_open = data_book[1172:234835+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(1,'02')
timestamp_time_second = time_transform(timestamp_time)

In [83]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 82.308868


In [84]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 166.693876


In [85]:
close_price = 7030
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [86]:
time1 = 0
time2 = 600
mat02_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [87]:
time1 = 600
time2 = 7200
mat02_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [88]:
time1 = 7200
time2 = 14400
mat02_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [89]:
time1 = 14400
time2 = 25200
mat02_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat02_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [91]:
#mat02_3_16

In [165]:
#mat17_2_16

In [92]:
SUM_Ask1_15 = \
array(mat02_1_15)+array(mat16_1_15)+array(mat17_1_15)+array(mat20_1_15)+array(mat21_1_15)+array(mat22_1_15)+\
array(mat24_1_15)+array(mat27_1_15)+array(mat28_1_15)+array(mat29_1_15)
SUM_Ask2_15 = \
array(mat02_2_15)+array(mat16_2_15)+array(mat17_2_15)+array(mat20_2_15)+array(mat21_2_15)+array(mat22_2_15)+\
array(mat24_2_15)+array(mat27_2_15)+array(mat28_2_15)+array(mat29_2_15)
SUM_Ask3_15 = \
array(mat02_3_15)+array(mat16_3_15)+array(mat17_3_15)+array(mat20_3_15)+array(mat21_3_15)+array(mat22_3_15)+\
array(mat24_3_15)+array(mat27_3_15)+array(mat28_3_15)+array(mat29_3_15)

In [170]:
#SUM_Bid3_15

In [93]:
SUM_Ask1_11 = \
array(mat02_1_11)+array(mat16_1_11)+array(mat17_1_11)+array(mat20_1_11)+array(mat21_1_11)+array(mat22_1_11)+\
array(mat24_1_11)+array(mat27_1_11)+array(mat28_1_11)+array(mat29_1_11)
SUM_Ask2_11 = \
array(mat02_2_11)+array(mat16_2_11)+array(mat17_2_11)+array(mat20_2_11)+array(mat21_2_11)+array(mat22_2_11)+\
array(mat24_2_11)+array(mat27_2_11)+array(mat28_2_11)+array(mat29_2_11)
SUM_Ask3_11 = \
array(mat02_3_11)+array(mat16_3_11)+array(mat17_3_11)+array(mat20_3_11)+array(mat21_3_11)+array(mat22_3_11)+\
array(mat24_3_11)+array(mat27_3_11)+array(mat28_3_11)+array(mat29_3_11)

In [175]:
#SUM_Bid3_11

In [94]:
SUM_Ask1_13 = \
array(mat02_1_13)+array(mat16_1_13)+array(mat17_1_13)+array(mat20_1_13)+array(mat21_1_13)+array(mat22_1_13)+\
array(mat24_1_13)+array(mat27_1_13)+array(mat28_1_13)+array(mat29_1_13)
SUM_Ask2_13 = \
array(mat02_2_13)+array(mat16_2_13)+array(mat17_2_13)+array(mat20_2_13)+array(mat21_2_13)+array(mat22_2_13)+\
array(mat24_2_13)+array(mat27_2_13)+array(mat28_2_13)+array(mat29_2_13)
SUM_Ask3_13 = \
array(mat02_3_13)+array(mat16_3_13)+array(mat17_3_13)+array(mat20_3_13)+array(mat21_3_13)+array(mat22_3_13)+\
array(mat24_3_13)+array(mat27_3_13)+array(mat28_3_13)+array(mat29_3_13)

In [180]:
#SUM_Bid3_13

In [95]:
SUM_Ask1_16 = \
array(mat02_1_16)+array(mat16_1_16)+array(mat17_1_16)+array(mat20_1_16)+array(mat21_1_16)+array(mat22_1_16)+\
array(mat24_1_16)+array(mat27_1_16)+array(mat28_1_16)+array(mat29_1_16)
SUM_Ask2_16 = \
array(mat02_2_16)+array(mat16_2_16)+array(mat17_2_16)+array(mat20_2_16)+array(mat21_2_16)+array(mat22_2_16)+\
array(mat24_2_16)+array(mat27_2_16)+array(mat28_2_16)+array(mat29_2_16)
SUM_Ask3_16 = \
array(mat02_3_16)+array(mat16_3_16)+array(mat17_3_16)+array(mat20_3_16)+array(mat21_3_16)+array(mat22_3_16)+\
array(mat24_3_16)+array(mat27_3_16)+array(mat28_3_16)+array(mat29_3_16)

In [185]:
#SUM_Bid3_16

In [96]:
a = 0
Ask1_15_1 = pd.DataFrame([SUM_Ask1_15[a],SUM_Ask1_15[a+1]/SUM_Ask1_15[a],SUM_Ask1_15[a+2]/SUM_Ask1_15[a],SUM_Ask1_15[a+3]/SUM_Ask1_15[a],SUM_Ask1_15[a+4]/SUM_Ask1_15[a]])

In [97]:
Ask1_15_1

,0,1,2,3,4
0,305.000000,346.000000,3002.000000,382.000000,98.000000
1,0.547541,0.604046,0.392738,0.256545,0.000000
2,-2.216393,-0.794798,-2.924384,-3.565445,-7.979592
3,5.724590,5.578035,6.392405,8.730366,10.000000
4,834.816393,663.731214,720.077282,824.947644,0.000000


In [98]:
a = 5
Ask1_15_2 = pd.DataFrame([SUM_Ask1_15[a],SUM_Ask1_15[a+1]/SUM_Ask1_15[a],SUM_Ask1_15[a+2]/SUM_Ask1_15[a],SUM_Ask1_15[a+3]/SUM_Ask1_15[a],SUM_Ask1_15[a+4]/SUM_Ask1_15[a]])

In [99]:
Ask1_15_2

,0,1,2,3,4
0,408.000000,493.000000,943.000000,143.000000,9.000000
1,0.350490,0.626775,0.417815,0.454545,1.000000
2,-3.549020,-1.279919,-3.884411,-2.965035,1.000000
3,6.987745,5.675456,7.262990,6.874126,5.000000
4,652.237745,540.797160,612.732768,602.237762,213.444444


In [101]:
a = 10
Ask1_15_3 = pd.DataFrame([SUM_Ask1_15[a],SUM_Ask1_15[a+1]/SUM_Ask1_15[a],SUM_Ask1_15[a+2]/SUM_Ask1_15[a],SUM_Ask1_15[a+3]/SUM_Ask1_15[a],SUM_Ask1_15[a+4]/SUM_Ask1_15[a]])

In [102]:
Ask1_15_3

,0,1,2,3,4
0,340.000000,461.000000,859.000000,115.000000,0.0
1,0.288235,0.694143,0.604191,0.565217,NaN
2,-4.473529,-0.550976,-1.263097,-1.252174,NaN
3,8.117647,5.013015,5.110594,5.791304,NaN
4,717.652941,465.763557,530.689173,530.113043,NaN


In [103]:
a = 15
Ask1_15_4 = pd.DataFrame([SUM_Ask1_15[a],SUM_Ask1_15[a+1]/SUM_Ask1_15[a],SUM_Ask1_15[a+2]/SUM_Ask1_15[a],SUM_Ask1_15[a+3]/SUM_Ask1_15[a],SUM_Ask1_15[a+4]/SUM_Ask1_15[a]])

In [104]:
Ask1_15_4

,0,1,2,3,4
0,188.000000,241.000000,622.000000,84.000000,0.0
1,0.367021,0.647303,0.602894,0.273810,NaN
2,-4.287234,-1.597510,-1.572347,-4.630952,NaN
3,7.526596,5.477178,5.151125,8.142857,NaN
4,636.031915,441.174274,517.940514,720.964286,NaN


In [105]:
a = 20
Ask1_15_5 = pd.DataFrame([SUM_Ask1_15[a],SUM_Ask1_15[a+1]/SUM_Ask1_15[a],SUM_Ask1_15[a+2]/SUM_Ask1_15[a],SUM_Ask1_15[a+3]/SUM_Ask1_15[a],SUM_Ask1_15[a+4]/SUM_Ask1_15[a]])

In [106]:
Ask1_15_5

,0,1,2,3,4
0,402.000000,517.000000,1578.000000,462.000000,2.0
1,0.308458,0.702128,0.726869,0.623377,0.0
2,-5.101990,-0.978723,-0.657161,-1.474026,-1.5
3,8.457711,4.597679,4.504436,4.439394,7.0
4,670.054726,389.444874,450.773764,443.170996,0.0


In [107]:
pd.concat([Ask1_15_1,Ask1_15_2,Ask1_15_3,Ask1_15_4,Ask1_15_5])

,0,1,2,3,4
0,305.000000,346.000000,3002.000000,382.000000,98.000000
1,0.547541,0.604046,0.392738,0.256545,0.000000
2,-2.216393,-0.794798,-2.924384,-3.565445,-7.979592
3,5.724590,5.578035,6.392405,8.730366,10.000000
4,834.816393,663.731214,720.077282,824.947644,0.000000
0,408.000000,493.000000,943.000000,143.000000,9.000000
1,0.350490,0.626775,0.417815,0.454545,1.000000
2,-3.549020,-1.279919,-3.884411,-2.965035,1.000000
3,6.987745,5.675456,7.262990,6.874126,5.000000
4,652.237745,540.797160,612.732768,602.237762,213.444444


In [108]:
pd.concat([Ask1_15_1,Ask1_15_2,Ask1_15_3,Ask1_15_4,Ask1_15_5]).to_csv('ask1_15_115_102.csv')

In [109]:
a = 0
Ask2_15_1 = pd.DataFrame([SUM_Ask2_15[a],SUM_Ask2_15[a+1]/SUM_Ask2_15[a],SUM_Ask2_15[a+2]/SUM_Ask2_15[a],SUM_Ask2_15[a+3]/SUM_Ask2_15[a],SUM_Ask2_15[a+4]/SUM_Ask2_15[a]])

In [110]:
a = 5
Ask2_15_2 = pd.DataFrame([SUM_Ask2_15[a],SUM_Ask2_15[a+1]/SUM_Ask2_15[a],SUM_Ask2_15[a+2]/SUM_Ask2_15[a],SUM_Ask2_15[a+3]/SUM_Ask2_15[a],SUM_Ask2_15[a+4]/SUM_Ask2_15[a]])

In [111]:
a = 10
Ask2_15_3 = pd.DataFrame([SUM_Ask2_15[a],SUM_Ask2_15[a+1]/SUM_Ask2_15[a],SUM_Ask2_15[a+2]/SUM_Ask2_15[a],SUM_Ask2_15[a+3]/SUM_Ask2_15[a],SUM_Ask2_15[a+4]/SUM_Ask2_15[a]])

In [112]:
a = 15
Ask2_15_4 = pd.DataFrame([SUM_Ask2_15[a],SUM_Ask2_15[a+1]/SUM_Ask2_15[a],SUM_Ask2_15[a+2]/SUM_Ask2_15[a],SUM_Ask2_15[a+3]/SUM_Ask2_15[a],SUM_Ask2_15[a+4]/SUM_Ask2_15[a]])

In [113]:
a = 20
Ask2_15_5 = pd.DataFrame([SUM_Ask2_15[a],SUM_Ask2_15[a+1]/SUM_Ask2_15[a],SUM_Ask2_15[a+2]/SUM_Ask2_15[a],SUM_Ask2_15[a+3]/SUM_Ask2_15[a],SUM_Ask2_15[a+4]/SUM_Ask2_15[a]])

In [114]:
pd.concat([Ask2_15_1,Ask2_15_2,Ask2_15_3,Ask2_15_4,Ask2_15_5])

,0,1,2,3,4
0,305.000000,346.000000,3002.000000,382.000000,98.000000
1,0.459016,0.488439,0.131246,0.060209,0.000000
2,-2.068852,-0.497110,-3.222518,-4.806283,-7.979592
3,7.062295,6.748555,7.872751,9.971204,11.000000
4,945.478689,712.549133,844.343105,873.366492,0.000000
0,408.000000,493.000000,943.000000,143.000000,9.000000
1,0.205882,0.578093,0.232238,0.356643,1.000000
2,-3.901961,-0.880325,-4.292683,-3.069930,2.000000
3,8.647059,6.971602,8.869565,8.601399,6.000000
4,788.379902,592.947262,766.735949,677.783217,216.444444


In [115]:
pd.concat([Ask2_15_1,Ask2_15_2,Ask2_15_3,Ask2_15_4,Ask2_15_5]).to_csv('ask2_15_115_102.csv')

In [116]:
a = 0
Ask3_15_1 = pd.DataFrame([SUM_Ask3_15[a],SUM_Ask3_15[a+1]/SUM_Ask3_15[a],SUM_Ask3_15[a+2]/SUM_Ask3_15[a],SUM_Ask3_15[a+3]/SUM_Ask3_15[a],SUM_Ask3_15[a+4]/SUM_Ask3_15[a]])

In [117]:
a = 5
Ask3_15_2 = pd.DataFrame([SUM_Ask3_15[a],SUM_Ask3_15[a+1]/SUM_Ask3_15[a],SUM_Ask3_15[a+2]/SUM_Ask3_15[a],SUM_Ask3_15[a+3]/SUM_Ask3_15[a],SUM_Ask3_15[a+4]/SUM_Ask3_15[a]])

In [118]:
a = 10
Ask3_15_3 = pd.DataFrame([SUM_Ask3_15[a],SUM_Ask3_15[a+1]/SUM_Ask3_15[a],SUM_Ask3_15[a+2]/SUM_Ask3_15[a],SUM_Ask3_15[a+3]/SUM_Ask3_15[a],SUM_Ask3_15[a+4]/SUM_Ask3_15[a]])

In [119]:
a = 15
Ask3_15_4 = pd.DataFrame([SUM_Ask3_15[a],SUM_Ask3_15[a+1]/SUM_Ask3_15[a],SUM_Ask3_15[a+2]/SUM_Ask3_15[a],SUM_Ask3_15[a+3]/SUM_Ask3_15[a],SUM_Ask3_15[a+4]/SUM_Ask3_15[a]])

In [120]:
a = 20
Ask3_15_5 = pd.DataFrame([SUM_Ask3_15[a],SUM_Ask3_15[a+1]/SUM_Ask3_15[a],SUM_Ask3_15[a+2]/SUM_Ask3_15[a],SUM_Ask3_15[a+3]/SUM_Ask3_15[a],SUM_Ask3_15[a+4]/SUM_Ask3_15[a]])

In [121]:
pd.concat([Ask3_15_1,Ask3_15_2,Ask3_15_3,Ask3_15_4,Ask3_15_5])

,0,1,2,3,4
0,305.000000,346.000000,3002.000000,382.000000,98.000000
1,0.308197,0.115607,0.099933,0.026178,0.000000
2,-1.908197,-1.127168,-3.225849,-5.086387,-7.979592
3,8.127869,7.760116,8.922385,11.107330,12.000000
4,1047.737705,832.193642,869.523984,892.811518,0.000000
0,408.000000,493.000000,943.000000,143.000000,9.000000
1,0.132353,0.176471,0.121951,0.153846,0.000000
2,-3.848039,-1.184584,-4.612937,-4.762238,-7.000000
3,9.678922,8.056795,10.065748,10.412587,11.000000
4,851.811275,791.387424,844.670201,802.419580,0.000000


In [122]:
pd.concat([Ask3_15_1,Ask3_15_2,Ask3_15_3,Ask3_15_4,Ask3_15_5]).to_csv('ask3_15_115_102.csv')

In [123]:
a = 0
Ask1_11_1 = pd.DataFrame([SUM_Ask1_11[a],SUM_Ask1_11[a+1]/SUM_Ask1_11[a],SUM_Ask1_11[a+2]/SUM_Ask1_11[a],SUM_Ask1_11[a+3]/SUM_Ask1_11[a],SUM_Ask1_11[a+4]/SUM_Ask1_11[a]])

In [124]:
a = 5
Ask1_11_2 = pd.DataFrame([SUM_Ask1_11[a],SUM_Ask1_11[a+1]/SUM_Ask1_11[a],SUM_Ask1_11[a+2]/SUM_Ask1_11[a],SUM_Ask1_11[a+3]/SUM_Ask1_11[a],SUM_Ask1_11[a+4]/SUM_Ask1_11[a]])

In [125]:
a = 10
Ask1_11_3 = pd.DataFrame([SUM_Ask1_11[a],SUM_Ask1_11[a+1]/SUM_Ask1_11[a],SUM_Ask1_11[a+2]/SUM_Ask1_11[a],SUM_Ask1_11[a+3]/SUM_Ask1_11[a],SUM_Ask1_11[a+4]/SUM_Ask1_11[a]])

In [126]:
a = 15
Ask1_11_4 = pd.DataFrame([SUM_Ask1_11[a],SUM_Ask1_11[a+1]/SUM_Ask1_11[a],SUM_Ask1_11[a+2]/SUM_Ask1_11[a],SUM_Ask1_11[a+3]/SUM_Ask1_11[a],SUM_Ask1_11[a+4]/SUM_Ask1_11[a]])

In [127]:
a = 20
Ask1_11_5 = pd.DataFrame([SUM_Ask1_11[a],SUM_Ask1_11[a+1]/SUM_Ask1_11[a],SUM_Ask1_11[a+2]/SUM_Ask1_11[a],SUM_Ask1_11[a+3]/SUM_Ask1_11[a],SUM_Ask1_11[a+4]/SUM_Ask1_11[a]])

In [128]:
pd.concat([Ask1_11_1,Ask1_11_2,Ask1_11_3,Ask1_11_4,Ask1_11_5])

,0,1,2,3,4
0,668.000000,2242.000000,5021.000000,840.000000,625.000000
1,0.583832,0.331401,0.512249,0.630952,0.547200
2,-0.467066,-2.165477,-1.392551,-1.054762,-1.460800
3,5.432635,5.574041,4.740092,4.591667,4.489600
4,1014.568862,681.876896,573.552280,459.502381,63.670400
0,699.000000,2327.000000,6631.000000,1105.000000,680.000000
1,0.625179,0.494628,0.601870,0.594570,0.686765
2,-0.349070,-1.078642,-0.836073,-0.586425,-0.641176
3,4.570815,4.162011,4.257126,4.200000,3.939706
4,825.821173,558.807907,530.772433,498.084163,113.066176


In [130]:
pd.concat([Ask1_11_1,Ask1_11_2,Ask1_11_3,Ask1_11_4,Ask1_11_5]).to_csv('ask1_11_115_102.csv')

In [131]:
a = 0
Ask2_11_1 = pd.DataFrame([SUM_Ask2_11[a],SUM_Ask2_11[a+1]/SUM_Ask2_11[a],SUM_Ask2_11[a+2]/SUM_Ask2_11[a],SUM_Ask2_11[a+3]/SUM_Ask2_11[a],SUM_Ask2_11[a+4]/SUM_Ask2_11[a]])

In [132]:
a = 5
Ask2_11_2 = pd.DataFrame([SUM_Ask2_11[a],SUM_Ask2_11[a+1]/SUM_Ask2_11[a],SUM_Ask2_11[a+2]/SUM_Ask2_11[a],SUM_Ask2_11[a+3]/SUM_Ask2_11[a],SUM_Ask2_11[a+4]/SUM_Ask2_11[a]])

In [133]:
a = 10
Ask2_11_3 = pd.DataFrame([SUM_Ask2_11[a],SUM_Ask2_11[a+1]/SUM_Ask2_11[a],SUM_Ask2_11[a+2]/SUM_Ask2_11[a],SUM_Ask2_11[a+3]/SUM_Ask2_11[a],SUM_Ask2_11[a+4]/SUM_Ask2_11[a]])

In [134]:
a = 15
Ask2_11_4 = pd.DataFrame([SUM_Ask2_11[a],SUM_Ask2_11[a+1]/SUM_Ask2_11[a],SUM_Ask2_11[a+2]/SUM_Ask2_11[a],SUM_Ask2_11[a+3]/SUM_Ask2_11[a],SUM_Ask2_11[a+4]/SUM_Ask2_11[a]])

In [135]:
a = 20
Ask2_11_5 = pd.DataFrame([SUM_Ask2_11[a],SUM_Ask2_11[a+1]/SUM_Ask2_11[a],SUM_Ask2_11[a+2]/SUM_Ask2_11[a],SUM_Ask2_11[a+3]/SUM_Ask2_11[a],SUM_Ask2_11[a+4]/SUM_Ask2_11[a]])

In [136]:
pd.concat([Ask2_11_1,Ask2_11_2,Ask2_11_3,Ask2_11_4,Ask2_11_5])

,0,1,2,3,4
0,668.000000,2242.000000,5021.000000,840.000000,625.000000
1,0.296407,0.172168,0.363872,0.401190,0.435200
2,-1.004491,-2.389831,-1.330014,-1.426190,-1.689600
3,7.007485,6.858608,6.030472,6.379762,6.113600
4,1266.965569,795.842105,702.037841,643.465476,68.996800
0,699.000000,2327.000000,6631.000000,1105.000000,680.000000
1,0.426323,0.312419,0.373850,0.436199,0.535294
2,-0.286123,-1.342501,-1.007691,-0.505882,-0.707353
3,5.882690,5.608509,5.762328,5.495023,5.575000
4,1088.752504,711.948861,707.085809,627.485973,114.285294


In [137]:
pd.concat([Ask2_11_1,Ask2_11_2,Ask2_11_3,Ask2_11_4,Ask2_11_5]).to_csv('ask2_11_115_102.csv')

In [138]:
a = 0
Ask3_11_1 = pd.DataFrame([SUM_Ask3_11[a],SUM_Ask3_11[a+1]/SUM_Ask3_11[a],SUM_Ask3_11[a+2]/SUM_Ask3_11[a],SUM_Ask3_11[a+3]/SUM_Ask3_11[a],SUM_Ask3_11[a+4]/SUM_Ask3_11[a]])

In [139]:
a = 5
Ask3_11_2 = pd.DataFrame([SUM_Ask3_11[a],SUM_Ask3_11[a+1]/SUM_Ask3_11[a],SUM_Ask3_11[a+2]/SUM_Ask3_11[a],SUM_Ask3_11[a+3]/SUM_Ask3_11[a],SUM_Ask3_11[a+4]/SUM_Ask3_11[a]])

In [140]:
a = 10
Ask3_11_3 = pd.DataFrame([SUM_Ask3_11[a],SUM_Ask3_11[a+1]/SUM_Ask3_11[a],SUM_Ask3_11[a+2]/SUM_Ask3_11[a],SUM_Ask3_11[a+3]/SUM_Ask3_11[a],SUM_Ask3_11[a+4]/SUM_Ask3_11[a]])

In [141]:
a = 15
Ask3_11_4 = pd.DataFrame([SUM_Ask3_11[a],SUM_Ask3_11[a+1]/SUM_Ask3_11[a],SUM_Ask3_11[a+2]/SUM_Ask3_11[a],SUM_Ask3_11[a+3]/SUM_Ask3_11[a],SUM_Ask3_11[a+4]/SUM_Ask3_11[a]])

In [142]:
a = 20
Ask3_11_5 = pd.DataFrame([SUM_Ask3_11[a],SUM_Ask3_11[a+1]/SUM_Ask3_11[a],SUM_Ask3_11[a+2]/SUM_Ask3_11[a],SUM_Ask3_11[a+3]/SUM_Ask3_11[a],SUM_Ask3_11[a+4]/SUM_Ask3_11[a]])

In [143]:
pd.concat([Ask3_11_1,Ask3_11_2,Ask3_11_3,Ask3_11_4,Ask3_11_5])

,0,1,2,3,4
0,668.000000,2242.000000,5021.000000,840.000000,625.000000
1,0.184132,0.126227,0.245967,0.334524,0.382400
2,-0.820359,-2.330508,-1.372237,-1.303571,-1.609600
3,8.011976,7.878680,7.180044,7.601190,7.395200
4,1332.934132,830.848350,776.100578,711.379762,81.297600
0,699.000000,2327.000000,6631.000000,1105.000000,680.000000
1,0.337625,0.220885,0.252752,0.344796,0.466176
2,0.032904,-1.258702,-0.941487,-0.322172,-0.608824
3,6.915594,6.679845,6.899563,6.646154,6.848529
4,1190.274678,784.220026,784.482883,720.619910,130.270588


In [144]:
pd.concat([Ask3_11_1,Ask3_11_2,Ask3_11_3,Ask3_11_4,Ask3_11_5]).to_csv('ask3_11_115_102.csv')

In [145]:
a = 0
Ask1_13_1 = pd.DataFrame([SUM_Ask1_13[a],SUM_Ask1_13[a+1]/SUM_Ask1_13[a],SUM_Ask1_13[a+2]/SUM_Ask1_13[a],SUM_Ask1_13[a+3]/SUM_Ask1_13[a],SUM_Ask1_13[a+4]/SUM_Ask1_13[a]])

In [146]:
a = 5
Ask1_13_2 = pd.DataFrame([SUM_Ask1_13[a],SUM_Ask1_13[a+1]/SUM_Ask1_13[a],SUM_Ask1_13[a+2]/SUM_Ask1_13[a],SUM_Ask1_13[a+3]/SUM_Ask1_13[a],SUM_Ask1_13[a+4]/SUM_Ask1_13[a]])

In [147]:
a = 10
Ask1_13_3 = pd.DataFrame([SUM_Ask1_13[a],SUM_Ask1_13[a+1]/SUM_Ask1_13[a],SUM_Ask1_13[a+2]/SUM_Ask1_13[a],SUM_Ask1_13[a+3]/SUM_Ask1_13[a],SUM_Ask1_13[a+4]/SUM_Ask1_13[a]])

In [148]:
a = 15
Ask1_13_4 = pd.DataFrame([SUM_Ask1_13[a],SUM_Ask1_13[a+1]/SUM_Ask1_13[a],SUM_Ask1_13[a+2]/SUM_Ask1_13[a],SUM_Ask1_13[a+3]/SUM_Ask1_13[a],SUM_Ask1_13[a+4]/SUM_Ask1_13[a]])

In [149]:
a = 20
Ask1_13_5 = pd.DataFrame([SUM_Ask1_13[a],SUM_Ask1_13[a+1]/SUM_Ask1_13[a],SUM_Ask1_13[a+2]/SUM_Ask1_13[a],SUM_Ask1_13[a+3]/SUM_Ask1_13[a],SUM_Ask1_13[a+4]/SUM_Ask1_13[a]])

In [150]:
pd.concat([Ask1_13_1,Ask1_13_2,Ask1_13_3,Ask1_13_4,Ask1_13_5])

,0,1,2,3,4
0,76.000000,218.000000,18626.000000,870.000000,95.000000
1,1.000000,0.330275,0.155052,0.127586,0.242105
2,1.000000,-0.362385,-0.085848,-0.996552,-1.821053
3,2.644737,2.188073,1.771019,2.875862,4.210526
4,1121.078947,675.000000,830.022657,825.860920,11.136842
0,55.000000,288.000000,14589.000000,580.000000,193.000000
1,1.000000,0.607639,0.068682,0.358621,0.248705
2,1.218182,0.375000,-0.171979,-0.915517,-1.683938
3,2.400000,2.041667,1.606896,3.491379,3.507772
4,2634.418182,497.135417,853.982932,692.334483,13.227979


In [151]:
pd.concat([Ask1_13_1,Ask1_13_2,Ask1_13_3,Ask1_13_4,Ask1_13_5]).to_csv('ask1_13_115_102.csv')

In [152]:
a = 0
Ask2_13_1 = pd.DataFrame([SUM_Ask2_13[a],SUM_Ask2_13[a+1]/SUM_Ask2_13[a],SUM_Ask2_13[a+2]/SUM_Ask2_13[a],SUM_Ask2_13[a+3]/SUM_Ask2_13[a],SUM_Ask2_13[a+4]/SUM_Ask2_13[a]])

In [153]:
a = 5
Ask2_13_2 =pd.DataFrame([SUM_Ask2_13[a],SUM_Ask2_13[a+1]/SUM_Ask2_13[a],SUM_Ask2_13[a+2]/SUM_Ask2_13[a],SUM_Ask2_13[a+3]/SUM_Ask2_13[a],SUM_Ask2_13[a+4]/SUM_Ask2_13[a]])

In [154]:
a = 10
Ask2_13_3 =pd.DataFrame([SUM_Ask2_13[a],SUM_Ask2_13[a+1]/SUM_Ask2_13[a],SUM_Ask2_13[a+2]/SUM_Ask2_13[a],SUM_Ask2_13[a+3]/SUM_Ask2_13[a],SUM_Ask2_13[a+4]/SUM_Ask2_13[a]])

In [155]:
a = 15
Ask2_13_4 =pd.DataFrame([SUM_Ask2_13[a],SUM_Ask2_13[a+1]/SUM_Ask2_13[a],SUM_Ask2_13[a+2]/SUM_Ask2_13[a],SUM_Ask2_13[a+3]/SUM_Ask2_13[a],SUM_Ask2_13[a+4]/SUM_Ask2_13[a]])

In [156]:
a = 20
Ask2_13_5 =pd.DataFrame([SUM_Ask2_13[a],SUM_Ask2_13[a+1]/SUM_Ask2_13[a],SUM_Ask2_13[a+2]/SUM_Ask2_13[a],SUM_Ask2_13[a+3]/SUM_Ask2_13[a],SUM_Ask2_13[a+4]/SUM_Ask2_13[a]])

In [157]:
pd.concat([Ask2_13_1,Ask2_13_2,Ask2_13_3,Ask2_13_4,Ask2_13_5])

,0,1,2,3,4
0,76.000000,218.000000,18626.000000,870.000000,95.000000
1,1.000000,0.238532,0.107323,0.085057,0.231579
2,2.000000,-0.279817,-0.015033,-1.002299,-1.631579
3,3.644737,3.215596,2.787448,3.921839,5.252632
4,1264.986842,722.151376,857.626597,855.654023,14.515789
0,55.000000,288.000000,14589.000000,580.000000,193.000000
1,0.945455,0.246528,0.042361,0.198276,0.222798
2,2.054545,0.350694,-0.166290,-1.082759,-1.538860
3,3.454545,3.177083,2.637124,4.736207,4.554404
4,2851.854545,742.423611,874.193570,800.043103,13.704663


In [158]:
pd.concat([Ask2_13_1,Ask2_13_2,Ask2_13_3,Ask2_13_4,Ask2_13_5]).to_csv('ask2_13_115_102.csv')

In [159]:
a = 0
Ask3_13_1 = pd.DataFrame([SUM_Ask3_13[a],SUM_Ask3_13[a+1]/SUM_Ask3_13[a],SUM_Ask3_13[a+2]/SUM_Ask3_13[a],SUM_Ask3_13[a+3]/SUM_Ask3_13[a],SUM_Ask3_13[a+4]/SUM_Ask3_13[a]])

In [160]:
a = 5
Ask3_13_2 = pd.DataFrame([SUM_Ask3_13[a],SUM_Ask3_13[a+1]/SUM_Ask3_13[a],SUM_Ask3_13[a+2]/SUM_Ask3_13[a],SUM_Ask3_13[a+3]/SUM_Ask3_13[a],SUM_Ask3_13[a+4]/SUM_Ask3_13[a]])

In [161]:
a = 10
Ask3_13_3 = pd.DataFrame([SUM_Ask3_13[a],SUM_Ask3_13[a+1]/SUM_Ask3_13[a],SUM_Ask3_13[a+2]/SUM_Ask3_13[a],SUM_Ask3_13[a+3]/SUM_Ask3_13[a],SUM_Ask3_13[a+4]/SUM_Ask3_13[a]])

In [162]:
a = 15
Ask3_13_4 = pd.DataFrame([SUM_Ask3_13[a],SUM_Ask3_13[a+1]/SUM_Ask3_13[a],SUM_Ask3_13[a+2]/SUM_Ask3_13[a],SUM_Ask3_13[a+3]/SUM_Ask3_13[a],SUM_Ask3_13[a+4]/SUM_Ask3_13[a]])

In [163]:
a = 20
Ask3_13_5 = pd.DataFrame([SUM_Ask3_13[a],SUM_Ask3_13[a+1]/SUM_Ask3_13[a],SUM_Ask3_13[a+2]/SUM_Ask3_13[a],SUM_Ask3_13[a+3]/SUM_Ask3_13[a],SUM_Ask3_13[a+4]/SUM_Ask3_13[a]])

In [164]:
pd.concat([Ask3_13_1,Ask3_13_2,Ask3_13_3,Ask3_13_4,Ask3_13_5])

,0,1,2,3,4
0,76.000000,218.000000,18626.000000,870.000000,95.000000
1,0.894737,0.004587,0.068184,0.021839,0.231579
2,2.763158,-0.435780,0.032750,-1.078161,-1.400000
3,4.644737,4.261468,3.790615,4.955172,6.252632
4,1312.486842,896.807339,873.741598,889.245977,16.705263
0,55.000000,288.000000,14589.000000,580.000000,193.000000
1,0.690909,0.052083,0.027212,0.139655,0.207254
2,2.472727,0.343750,-0.160121,-1.046552,-1.393782
3,4.454545,4.215278,3.642745,5.755172,5.601036
4,3077.490909,874.225694,886.200356,832.474138,16.388601


In [165]:
pd.concat([Ask3_13_1,Ask3_13_2,Ask3_13_3,Ask3_13_4,Ask3_13_5]).to_csv('ask3_13_115_102.csv')

In [166]:
a = 0
Ask1_16_1 = pd.DataFrame([SUM_Ask1_16[a],SUM_Ask1_16[a+1]/SUM_Ask1_16[a],SUM_Ask1_16[a+2]/SUM_Ask1_16[a],SUM_Ask1_16[a+3]/SUM_Ask1_16[a],SUM_Ask1_16[a+4]/SUM_Ask1_16[a]])

In [167]:
a = 5
Ask1_16_2 = pd.DataFrame([SUM_Ask1_16[a],SUM_Ask1_16[a+1]/SUM_Ask1_16[a],SUM_Ask1_16[a+2]/SUM_Ask1_16[a],SUM_Ask1_16[a+3]/SUM_Ask1_16[a],SUM_Ask1_16[a+4]/SUM_Ask1_16[a]])

In [169]:
a = 10
Ask1_16_3 = pd.DataFrame([SUM_Ask1_16[a],SUM_Ask1_16[a+1]/SUM_Ask1_16[a],SUM_Ask1_16[a+2]/SUM_Ask1_16[a],SUM_Ask1_16[a+3]/SUM_Ask1_16[a],SUM_Ask1_16[a+4]/SUM_Ask1_16[a]])

In [170]:
a = 15
Ask1_16_4 = pd.DataFrame([SUM_Ask1_16[a],SUM_Ask1_16[a+1]/SUM_Ask1_16[a],SUM_Ask1_16[a+2]/SUM_Ask1_16[a],SUM_Ask1_16[a+3]/SUM_Ask1_16[a],SUM_Ask1_16[a+4]/SUM_Ask1_16[a]])

In [171]:
a = 20
Ask1_16_5 = pd.DataFrame([SUM_Ask1_16[a],SUM_Ask1_16[a+1]/SUM_Ask1_16[a],SUM_Ask1_16[a+2]/SUM_Ask1_16[a],SUM_Ask1_16[a+3]/SUM_Ask1_16[a],SUM_Ask1_16[a+4]/SUM_Ask1_16[a]])

In [172]:
pd.concat([Ask1_16_1,Ask1_16_2,Ask1_16_3,Ask1_16_4,Ask1_16_5])

,0,1,2,3,4
0,2.000000,463.000000,20260.000000,2152.000000,357.000000
1,0.000000,0.291577,0.270780,0.347584,0.445378
2,-2.000000,-0.442765,-0.573820,-0.339498,0.644258
3,3.000000,3.038877,2.808835,2.618030,1.408964
4,31950.000000,799.190065,722.249901,670.065520,135.778711
0,8.000000,1077.000000,17933.000000,1635.000000,165.000000
1,0.875000,0.305478,0.366754,0.275229,0.618182
2,0.750000,-0.414113,-0.190989,-0.413456,0.036364
3,1.375000,2.962860,2.703619,2.815902,2.454545
4,6045.125000,724.149489,703.432833,729.913761,115.036364


In [173]:
pd.concat([Ask1_16_1,Ask1_16_2,Ask1_16_3,Ask1_16_4,Ask1_16_5]).to_csv('ask1_16_115_102.csv')

In [174]:
a = 0
Ask2_16_1 = pd.DataFrame([SUM_Ask2_16[a],SUM_Ask2_16[a+1]/SUM_Ask2_16[a],SUM_Ask2_16[a+2]/SUM_Ask2_16[a],SUM_Ask2_16[a+3]/SUM_Ask2_16[a],SUM_Ask2_16[a+4]/SUM_Ask2_16[a]])

In [175]:
a = 5
Ask2_16_2 = pd.DataFrame([SUM_Ask2_16[a],SUM_Ask2_16[a+1]/SUM_Ask2_16[a],SUM_Ask2_16[a+2]/SUM_Ask2_16[a],SUM_Ask2_16[a+3]/SUM_Ask2_16[a],SUM_Ask2_16[a+4]/SUM_Ask2_16[a]])

In [176]:
a = 10
Ask2_16_3 = pd.DataFrame([SUM_Ask2_16[a],SUM_Ask2_16[a+1]/SUM_Ask2_16[a],SUM_Ask2_16[a+2]/SUM_Ask2_16[a],SUM_Ask2_16[a+3]/SUM_Ask2_16[a],SUM_Ask2_16[a+4]/SUM_Ask2_16[a]])

In [177]:
a = 15
Ask2_16_4 = pd.DataFrame([SUM_Ask2_16[a],SUM_Ask2_16[a+1]/SUM_Ask2_16[a],SUM_Ask2_16[a+2]/SUM_Ask2_16[a],SUM_Ask2_16[a+3]/SUM_Ask2_16[a],SUM_Ask2_16[a+4]/SUM_Ask2_16[a]])

In [178]:
a = 20
Ask2_16_5 = pd.DataFrame([SUM_Ask2_16[a],SUM_Ask2_16[a+1]/SUM_Ask2_16[a],SUM_Ask2_16[a+2]/SUM_Ask2_16[a],SUM_Ask2_16[a+3]/SUM_Ask2_16[a],SUM_Ask2_16[a+4]/SUM_Ask2_16[a]])

In [179]:
pd.concat([Ask2_16_1,Ask2_16_2,Ask2_16_3,Ask2_16_4,Ask2_16_5])

,0,1,2,3,4
0,2.000000,463.000000,20260.000000,2152.000000,357.000000
1,0.000000,0.187905,0.099753,0.091078,0.372549
2,-2.000000,-0.272138,-0.416496,-0.213569,0.932773
3,4.000000,4.066955,3.858391,3.640799,2.470588
4,43650.000000,853.805616,793.405726,787.440056,139.591036
0,8.000000,1077.000000,17933.000000,1635.000000,165.000000
1,0.000000,0.168059,0.178888,0.125994,0.569697
2,0.000000,-0.315692,-0.026342,-0.346177,0.460606
3,3.875000,4.066852,3.776390,3.856269,3.521212
4,7650.000000,827.623027,805.547259,820.525994,136.660606


In [180]:
pd.concat([Ask2_16_1,Ask2_16_2,Ask2_16_3,Ask2_16_4,Ask2_16_5]).to_csv('ask2_16_115_102.csv')

In [181]:
a = 0
Ask3_16_1 = pd.DataFrame([SUM_Ask3_16[a],SUM_Ask3_16[a+1]/SUM_Ask3_16[a],SUM_Ask3_16[a+2]/SUM_Ask3_16[a],SUM_Ask3_16[a+3]/SUM_Ask3_16[a],SUM_Ask3_16[a+4]/SUM_Ask3_16[a]])

In [182]:
a = 5
Ask3_16_2 = pd.DataFrame([SUM_Ask3_16[a],SUM_Ask3_16[a+1]/SUM_Ask3_16[a],SUM_Ask3_16[a+2]/SUM_Ask3_16[a],SUM_Ask3_16[a+3]/SUM_Ask3_16[a],SUM_Ask3_16[a+4]/SUM_Ask3_16[a]])

In [183]:
a = 10
Ask3_16_3 = pd.DataFrame([SUM_Ask3_16[a],SUM_Ask3_16[a+1]/SUM_Ask3_16[a],SUM_Ask3_16[a+2]/SUM_Ask3_16[a],SUM_Ask3_16[a+3]/SUM_Ask3_16[a],SUM_Ask3_16[a+4]/SUM_Ask3_16[a]])

In [184]:
a = 15
Ask3_16_4 = pd.DataFrame([SUM_Ask3_16[a],SUM_Ask3_16[a+1]/SUM_Ask3_16[a],SUM_Ask3_16[a+2]/SUM_Ask3_16[a],SUM_Ask3_16[a+3]/SUM_Ask3_16[a],SUM_Ask3_16[a+4]/SUM_Ask3_16[a]])

In [185]:
a = 20
Ask3_16_5 = pd.DataFrame([SUM_Ask3_16[a],SUM_Ask3_16[a+1]/SUM_Ask3_16[a],SUM_Ask3_16[a+2]/SUM_Ask3_16[a],SUM_Ask3_16[a+3]/SUM_Ask3_16[a],SUM_Ask3_16[a+4]/SUM_Ask3_16[a]])

In [186]:
pd.concat([Ask3_16_1,Ask3_16_2,Ask3_16_3,Ask3_16_4,Ask3_16_5])

,0,1,2,3,4
0,2.000000,463.000000,20260.000000,2152.000000,357.000000
1,0.000000,0.079914,0.054393,0.030669,0.291317
2,-2.000000,-0.207343,-0.373425,-0.207993,0.924370
3,5.000000,5.066955,4.860711,4.640799,3.661064
4,56700.000000,886.470842,814.716140,817.110130,128.445378
0,8.000000,1077.000000,17933.000000,1635.000000,165.000000
1,0.000000,0.075209,0.081191,0.052599,0.290909
2,0.000000,-0.292479,0.095400,-0.313150,0.206061
3,4.875000,5.066852,4.785424,4.856269,4.763636
4,12825.000000,876.228412,842.997379,852.740061,52.581818


In [335]:
pd.concat([Ask3_16_1,Ask3_16_2,Ask3_16_3,Ask3_16_4,Ask3_16_5]).to_csv('ask3_16_115_102.csv')